In [1]:
import requests
import pandas as pd
import json
import mysql.connector
from datetime import datetime, timedelta, timezone
import time
import nvda_config
import twelvedata as td
from twelvedata import TDClient


In [2]:
# set up login
Cpassword = nvda_config.mysql_password
td = TDClient(apikey=nvda_config.twelve_api_nvda) 
base_url = 'https://api.twelvedata.com'


In [11]:
# Estab;ish MySQL object
mydb = mysql.connector.connect(
    host="localhost",   
    user="root",         
    password=Cpassword,  
    database="bailish_estate"   
)

mycursor = mydb.cursor()

# Twelve Data API parameters
symbol = ["NVDA", "SPX", "XAU/USD"]
interval = "1min"
outputsize = 5000 

In [12]:

request_count = 0
while request_count < 1:
        for sym in symbol:
                price_pdf = td.price(
                symbol=sym
                ).as_json()
                timestamp = datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%S")  # Current timestamp
                
                price_df = pd.DataFrame(price_pdf, index=["price"])

                data_to_insert = [(sym, timestamp, price_df["price"].values[0])]
                # Insert tick data into MySQL
                sql_time_series = """
                INSERT INTO live_price (symbol, datetime, price)
                VALUES (%s, %s, %s)
                """
                mycursor.executemany(sql_time_series, data_to_insert)
                mydb.commit()

                # Increment the request counter
        request_count += 1
        print("Requests elapsed: ", request_count, " at time:", datetime.now())
        # Sleep for a short period before fetching the next data
        time.sleep(59.75) # Adjust the sleep time as needed

        print("Program completed after 390 requests.")


Requests elapsed:  1  at time: 2024-08-06 19:02:03.476016
Program completed after 390 requests.


In [ ]:
print()

In [ ]:
symbol_data = []
price_data = []
datetime_data = []
for symbol in symbol:
    # Fetch real-time price
    price_pdf = td.price(symbol=symbol)
    price_json = price_pdf.as_json()
    price_df = pd.DataFrame(pric, index=["price"])
    price = price_df["price"][0]

    # Generate the current datetime string
    datetime_str = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # Append data to lists
    symbol_data.append(symbol)
    price_data.append(price)
    datetime_data.append(datetime_str)

# Create DataFrame
df = pd.DataFrame({"symbol": symbol_data, "price": price_data, "datetime": datetime_data})
print(df)

# Insert data into MySQL (optional)
sql_insert = """
INSERT INTO live_price (symbol, datetime, price)
VALUES (%s, %s, %s)
"""
for _, row in df.iterrows():
    mycursor.execute(sql_insert, tuple(row))
    mydb.commit()


In [ ]:

def get_earnings_data(ticker, outputsize=50,  start_date = None, end_date = None):
    """Fetches earnings data for a given stock symbol from Twelve Data.

    Args:
        symbol (str): The stock symbol (e.g., "AAPL", "NVDA").
        outputsize (int, optional): Number of earnings records to retrieve (default=50, max=5000).
        timeframe (str, optional): 

    Returns:
        pd.DataFrame: A pandas DataFrame containing the earnings data.
    """

    res = td.get_earnings(
        symbol=ticker,
        outputsize=outputsize, 
        start_date = start_date, 
        end_date = end_date

    )

    # Convert the response data to a Pandas DataFrame
    df = pd.DataFrame(res.as_json())

    return df


In [ ]:
# Date range for historical data (funky start date due to downloading first month as test batch)
end_date = datetime.now()
start_date = end_date - timedelta(days=390)  
nvda_earnings_1year = get_earnings_data("NVDA", 1000, start_date, end_date)

print(nvda_earnings_1year)

In [ ]:

nvda_earnings_1year.to_csv(symbol + '_earnings.csv', index = False)